In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o")
small_llm = ChatOpenAI(model="gpt-4o-mini")

In [3]:
from langchain_core.tools import tool

@tool
def add(a : int, b : int) -> int:
    """숫자 a와 b를 더합니다."""
    return a + b

@tool
def multiply(a : int, b : int) -> int:
    """숫자 a와 b를 곱합니다."""
    return a * b

In [4]:
# tool 데코레이트 함수를 수행하는 방법
add.invoke({'a' : 1, 'b' : 2})

3

In [5]:
llm_with_tools = llm.bind_tools([add, multiply])

In [6]:
query = '3 곱하기 5는?'

In [7]:
# tool 없이 llm으로 질의했을 때
small_llm.invoke(query)

AIMessage(content='3 곱하기 5는 15입니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 15, 'total_tokens': 26, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': None, 'id': 'chatcmpl-Bu4zKvij6wGz89sUkF4e0aC1l8XsI', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--c7849427-09bd-47fc-91d5-2b49a0046639-0', usage_metadata={'input_tokens': 15, 'output_tokens': 11, 'total_tokens': 26, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [8]:
# tool binding 후 llm으로 질의했을 때
result = llm_with_tools.invoke(query)

In [9]:
result.tool_calls

[{'name': 'multiply',
  'args': {'a': 3, 'b': 5},
  'id': 'call_2EZtBUfi91bx1pMc12puULve',
  'type': 'tool_call'}]

In [10]:
from typing import Sequence
from langchain_core.messages import AnyMessage, HumanMessage

human_message = HumanMessage(query)
message_list : Sequence[AnyMessage] = [human_message]

In [11]:
ai_message = llm_with_tools.invoke(message_list)

In [12]:
ai_message.tool_calls

[{'name': 'multiply',
  'args': {'a': 3, 'b': 5},
  'id': 'call_It1jTgxRqdh73XAj47CT1RkQ',
  'type': 'tool_call'}]

In [13]:
message_list.append(ai_message)

In [14]:
tool_message = multiply.invoke(ai_message.tool_calls[0])

In [15]:
message_list.append(tool_message)

In [16]:
llm_with_tools.invoke(message_list)

AIMessage(content='3 곱하기 5는 15입니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 12, 'prompt_tokens': 109, 'total_tokens': 121, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': None, 'id': 'chatcmpl-Bu4zT4B6miw01646R2x6KLIp7d52w', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--2add5c20-e5d6-413f-a0c1-3deccc41e757-0', usage_metadata={'input_tokens': 109, 'output_tokens': 12, 'total_tokens': 121, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})